# Data Quality Spy
A Fabric notebook to scan for data quality issues in semantic models across workspaces in Fabric tenant. This version so far only looks at [referential integrity (RI) violations](https://dax.tips/2019/11/28/clean-data-faster-reports/) in semantic models.

---

What this notebook does:
1. takes a list of workspaces;
2. loops over semantic models in those workspaces to read tables and list RI violations in relationships between tables;
3. tries to capture errors for incompatible semantic models (e.g. auto-generated models like workspace Usage Metrics, default semantic model of lakehouse);
4. loads results to a Pandas DataFrame;
5. writes Pandas DataFrame to CSV file in lakehouse.

---

Room for improvement:
- refactor Python code (use dictionary comprehensions instead of loops where possible, more robust and verbose exception handling, verify PEP compliance).
- read list of workspaces from API response or file. Tutorial for API calls by Kurt Buhler: https://data-goblins.com/power-bi/power-bi-api-python.
- leverage Spark by using Spark DataFrames instead of combination of Fabric DataFrames and Pandas DataFrames.
- write results to files in batches to avoid memory issues when scanning many models.

---

Ideas for expansion:
- column statistics (data type, size, cardinality, missingness, variance) from DAX queries
- query DMVs if possible
---

Last updated: 2023-11-25

Author: Ruben Van de Voorde, revision by Max Van Goethem

---

## Code

In [ ]:
# provide list of ids for workspaces to check semantic models in; can be found after https://app.powerbi.com/groups/ in url for workspace
# hardcoded here but possible to read from API response or file instead
wss = [
    '11111111-1111-1111-1111-111111111111',
    '22222222-2222-2222-2222-222222222222',
    '33333333-3333-3333-3333-333333333333'
]

In [ ]:
# install dependencies
# consider installing semantic-link for workspace instead of %pip install
%pip install semantic-link
import pandas as pd
import datetime
import sempy.fabric as fabric
from sempy.fabric import list_relationship_violations

In [ ]:
# loop over semantic models to read tables and list RI violations in relationships between tables and try to capture errors
ws_violations = []
for ws in wss:
    dss = fabric.list_datasets(ws)['Dataset Name']
    for ds in dss:
        try:
            tbls = {tbl: fabric.read_table(table=tbl, dataset=ds, workspace=ws)
            for tbl in fabric.list_tables(dataset=ds, workspace=ws)['Name']}
            ds_violations = fabric.list_relationship_violations(tbls)
            ds_violations['Workspace Id'] = ws
            ds_violations['Dataset Name'] = ds
            ds_violations['Timestamp'] = datetime.datetime.now()
            ds_violations['Error Status'] = 'No error'
            ws_violations.append(ds_violations)
        except Exception as e:
            # need to create schema for errors df in case first dataset checked returns errors
            ds_errors = pd.DataFrame(columns=['Multiplicity', 'From Table', 'From Column', 'To Table',
            'To Column', 'Type', 'Message', 'Workspace Id', 'Dataset Name', 'Timestamp', 'Error Status'])
            ds_errors['Message'] = [e]
            ds_errors['Workspace Id'] = ws
            ds_errors['Dataset Name'] = ds
            ds_errors['Timestamp'] = datetime.datetime.now()
            ds_errors['Error Status'] = 'Error'
            ws_violations.append(ds_errors)

ri_violations = pd.concat(ws_violations)

In [ ]:
# print Pandas DataFrame for visual inspection
ri_violations

In [ ]:
# write Pandas DataFrame to CSV in lakehouse
ri_violations.to_csv(
    # can write to non-attached lakehouse by replacing "/lakehouse/default/" with ABFFS path
    # ABFFS path can be found in Properties of 'Files' in lakehouse UI
    "/lakehouse/default/"
    "/Files" +
    "/dataset-quality_ri-violations_" +
    str(datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")) +
    ".csv",
    index = False
)